In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import Core.Nanoparticle as NP
import Core.GlobalFeatureClassifier as GFC

from LocalOpt.LocalOptimization import local_optimization
import Core.EnergyCalculator as EC

from Core.LocalEnvironmentFeatureClassifier import TopologicalEnvironmentClassifier
from Core.LocalEnvironmentCalculator import LocalEnvironmentCalculator

from MCMC import MonteCarlo

from ase.visualize import view
import pickle

In [3]:
def create_octahedron_training_set(n_particles, height, trunc, stoichiometry):
    emt_calculator = EC.EMTCalculator(fmax=0.03, steps=1000)
    top = GFC.testTopologicalFeatureClassifier(list(stoichiometry.keys()))
    training_set = []
    for i in range(n_particles):
        p = NP.Nanoparticle()
        p.truncated_octahedron(height, trunc, stoichiometry)
        emt_calculator.compute_energy(p)
        training_set.append(p)
        top.compute_feature_vector(p)
        
        
    return training_set

In [4]:
def create_start_particle(height, trunc, stoichiometry):
    start_particle = NP.Nanoparticle()
    start_particle.truncated_octahedron(height, trunc, stoichiometry)
    return start_particle

In [5]:
"""Create the training set with 30 particles"""
stoichiometry={'Pt' : 56, 'Au' : 84}

training_set = create_octahedron_training_set(2, 6, 1, stoichiometry)

In [6]:
ordering_energy_calculator = EC.BayesianRRCalculator('TFC')
ordering_energy_calculator.fit(training_set, 'EMT')
top_coef = ordering_energy_calculator.get_coefficients()

coefficients, _ = EC.compute_coefficients_for_linear_topological_model(top_coef, training_set[1].get_all_symbols(), training_set[1].get_n_atoms()) 
ordering_energy_calculator.set_coefficients(coefficients)
ordering_energy_calculator.set_feature_key('TEC')
ordering_energy_calculator.energy_key = 'ORD'

Coef symbol_a: Au


In [7]:
adsorbate_calculator = EC.BayesianRRCalculator('ADS')
coeff_ads = [0, 0,
             -0.28120172, -1.01410455, -2.23349599,
             -0.35726243, -1.00039431, -2.08995281, -2.40469383]
adsorbate_calculator.set_coefficients(coeff_ads)

In [8]:
local_environment_calculator = LocalEnvironmentCalculator()
local_feature_classifier = TopologicalEnvironmentClassifier(
                                                local_environment_calculator,
                                                ['Au', 'Pt'])

In [11]:
start_particle = create_start_particle(6, 0, {'Au' : 0.6, 'Pt' : 0.4})
kwargs = {
    'beta' : 200,
    'max_steps' : 200,
    'start_particle' : start_particle,
    'ordering_energy_calculator' : ordering_energy_calculator, 
    'adsorbates_energy_calculator' : adsorbate_calculator,
    'n_adsorbates' : 5, 
    'local_feature_classifier' : local_feature_classifier}

[best_particle, accepted_energies, best_particles] = MonteCarlo.run_monte_carlo_ordering_adsorbates(**kwargs)

Resulting stoichiometry: {'Au': 88, 'Pt': 58}


In [75]:
from ase.io import write, read

atoms = read('PtAuO12.xyz')
renderer = write('PtAuO12.pov', atoms,
                 rotation='90z,-75x',
                 show_unit_cell=0,
                 povray_settings=dict(
                     pause=False,
                     canvas_width=1024,
                     camera_type='perspective',))
​
renderer.render()

SyntaxError: invalid non-printable character U+200B (2978548119.py, line 11)

In [12]:
from Core import Adsorption
from ase.visualize import view

ads_placer = Adsorption.PlaceAddAtoms(start_particle.get_all_symbols())
ads_placer.bind_particle(start_particle)
ads_site_list = start_particle.get_adsorption_list()

ads_positions = [list(ads_site_list[site]) for site in best_particle['ads']] 
ads_placer.place_add_atom(start_particle, 'O', ads_positions)
view(start_particle.get_ase_atoms(), viewer='x3d')